In [190]:
import pandas as pd
from matplotlib import pyplot as plt
import numpy as np
import seaborn as sns
import operator


In [28]:
df_users = pd.read_csv('users.csv')
df_users = df_users.drop_duplicates(subset='username')
df_users.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4223 entries, 0 to 4467
Data columns (total 7 columns):
_id                       4223 non-null object
username                  4223 non-null object
uid                       4223 non-null object
attraction_review_name    4223 non-null object
nb_cities_visited         4223 non-null object
src                       4223 non-null int64
nb_contributions          4223 non-null object
dtypes: int64(1), object(6)
memory usage: 263.9+ KB


In [56]:
df_reviews = pd.read_excel('reviews_32618_for_1098_users_with_location.xlsx')
df_reviews.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32580 entries, 0 to 32579
Data columns (total 12 columns):
id              32580 non-null int64
username        32580 non-null object
type            32580 non-null object
date            32580 non-null datetime64[ns]
title           32580 non-null object
text            32579 non-null object
rating          32580 non-null int64
helpfulness     24105 non-null float64
total_points    32580 non-null int64
taObject        32580 non-null object
taObjectUrl     32580 non-null object
taObjectCity    32580 non-null object
dtypes: datetime64[ns](1), float64(1), int64(3), object(7)
memory usage: 3.0+ MB


In [61]:
df = df_reviews.pivot_table(index='username',columns=['taObjectCity'],values='rating',aggfunc=np.mean)

In [103]:
df = df.fillna(0)
df.info()
df.iloc[[103],:]

<class 'pandas.core.frame.DataFrame'>
Index: 1098 entries, 422 to Farnborough
Columns: 5927 entries, Aachen to Г€ze
dtypes: float64(5927)
memory usage: 49.7+ MB


taObjectCity,Aachen,Aamby Valley City,Aarhus,Abbots Salford,Abbotsford,Aberaeron,Abercraf,Aberdeen,Aberdyfi (Aberdovey),Aberfeldy,...,Zipaquira,Zippori,Zoeterwoude,Zonnebeke,Zrece,Zurich,Zurs,Zwolle,Г–stersund,Г€ze
username,,,,,,,,,,,,,,,,,,,,,
ASWh,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0


In [63]:
count_list = []
for i in df.columns:
    count_list.append(df[i].sum())
print(np.mean(count_list))

12.648322906435409


In [65]:
df.head()

taObjectCity,Aachen,Aamby Valley City,Aarhus,Abbots Salford,Abbotsford,Aberaeron,Abercraf,Aberdeen,Aberdyfi (Aberdovey),Aberfeldy,...,Zipaquira,Zippori,Zoeterwoude,Zonnebeke,Zrece,Zurich,Zurs,Zwolle,Г–stersund,Г€ze
username,,,,,,,,,,,,,,,,,,,,,
422,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1771,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2093,0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5.0
4038,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5264,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [166]:
df.loc[((df.loc[:,:]*df.loc["ASWh",:]).replace(0,np.nan).count(axis='columns')>1),:].info()

<class 'pandas.core.frame.DataFrame'>
Index: 488 entries, 422 to FanMan10
Columns: 5927 entries, Aachen to Г€ze
dtypes: float64(5927)
memory usage: 22.1+ MB


In [275]:
def find_n_nearest_neighbors(username,n=5):
    dict_neighbours = {}
    df_common = df.loc[((df.loc[:,:]*df.loc[username,:]).replace(0,np.nan).count(axis='columns')>1),:]
    df_common = df_common.loc[:,df_common.loc[username,df_common.loc[username,:]>0].index]
    for val,row in df_common.iterrows():
        set_rated = df_common.loc[[val,username],(df_common.loc[val,:]*df_common.loc[username,:])>0].columns
        sim = np.sum(df_common.loc[val,set_rated]*df_common.loc[username,set_rated])/np.sqrt((np.sum(df_common.loc[val,set_rated]**2))*(np.sum(df_common.loc[username,set_rated]**2)))
        dict_neighbours[val] = sim
    return dict_neighbours

In [181]:
dict_neighbours = find_n_nearest_neighbors('ASWh')

In [182]:
for key,item in dict_neighbours.items():
    print(str(key) + ' : ' + str(item))

422 : 0.32411613667785893
2093 : 0.2513399233521418
243345 : 0.48245614035087714
678855 : 0.20211993611151446
007solotraveler : 0.33499170812603646
10179k79 : 0.49450549450549447
124_10 : 0.5037037037037037
1969ws : 0.5073891625615764
1975mark : 0.5
1984travel : 0.3338834791219673
19katy80 : 0.25387205387205386
1Angie : 0.5
1NicePerson : 0.16655415811929397
1Vegas2008 : 0.507936507936508
2008travellover : 0.33295838020247465
2011ChgoWanderer : 0.5006587615283268
23betty : 0.4962962962962963
25YearsAbroad : 0.5037037037037037
29grapes : 0.2001870816947571
2Travelers2go : 0.1456931936948597
2toots : 0.24823529411764703
32369Terry : 0.5
3kidsmama : 0.32793522267206476
45Jen : 0.5037037037037037
4959keeton : 0.33458775230927124
54Penny : 0.5
6079max : 0.20014900289342824
6dogs : 0.4837092731829575
7ContinentsX : 0.33393829401088926
A.Brouwer : 0.14285132216925395
AA75 : 0.5365853658536586
AANetherlands : 0.3333333333333333
ABCDEFG_1234 : 0.34152102031176196
ABitSpoiled : 0.4972527472527472

In [307]:
df_pers = pd.read_excel('pers_scores_1098.xlsx',index_col = 'username')
df_pers = df_pers.subtract(df_pers.mean(axis=0),axis=1).divide(df_pers.std(axis=0))
df_pers_neighbours = df_pers.loc[dict_neighbours.keys(),:]
df_pers_neighbours = (df_pers_neighbours.loc['ASWh',:]*df_pers_neighbours).divide(np.sqrt((np.sum(df_pers_neighbours.loc['ASWh',:]**2))*((df_pers_neighbours**2).sum(axis=1))),axis=0).sum(axis=1)

In [309]:
df_sim = (pd.Series(dict_neighbours) + df_pers_neighbours)/2
df_sim

422                   0.193097
2093                  0.250200
243345                0.348740
678855               -0.163190
007solotraveler       0.179903
10179k79              0.488399
124_10                0.114465
1969ws               -0.204070
1975mark              0.622019
1984travel            0.146838
19katy80              0.033707
1Angie               -0.029465
1NicePerson           0.135268
1Vegas2008            0.689500
2008travellover       0.609047
2011ChgoWanderer      0.691474
23betty               0.621158
25YearsAbroad         0.157656
29grapes              0.382518
2Travelers2go        -0.049192
2toots               -0.242533
32369Terry           -0.154188
3kidsmama             0.024963
45Jen                 0.657485
4959keeton            0.332259
54Penny               0.548063
6079max               0.441910
6dogs                -0.228594
7ContinentsX          0.319796
A.Brouwer            -0.235180
                        ...   
Erinluvsire          -0.042393
Ervaan  

In [195]:
print(df.loc[['Dee-ward','ASWh'],(df.loc['Dee-ward',:]*df.loc['ASWh',:])>0])

taObjectCity  London  New York City
username                           
Dee-ward         2.0       5.000000
ASWh             3.5       4.666667


In [319]:
sorted_df_sim = df_sim.sort_values(ascending=False)[:10]
sorted_df_sim

Chris_Jetten        0.735382
Dougdog             0.722737
ClaireUK87          0.722003
ARKJ14              0.719779
Anna1834            0.718277
Chunkymicky         0.714230
Charlemaine         0.703070
ClareKEssex         0.701063
BMBeer              0.700961
2011ChgoWanderer    0.691474
dtype: float64

In [321]:
neighbours = sorted_df_sim.index
sims = sorted_df_sim.values
condition = np.logical_or(df.loc[neighbours[1],:]>0,df.loc[neighbours[2],:]>0)
for i in range(3,len(neighbours)-1):
    condition = np.logical_or(condition,df.loc[neighbours[i],:]>0)
df_neighbours = df.loc[neighbours,condition]
df_neighbours = df_neighbours.replace(0,np.nan)
df_prediction = df.loc['ASWh',:].replace(0,np.nan).mean() + df_neighbours.sub(df_neighbours.mean(axis='columns'),axis=0).multiply(sims,axis=0).sum().divide(df_neighbours.multiply(sims,axis=0).sum())
df_prediction.sort_values(ascending=False)

taObjectCity
Dover                     4.181614
Warminster                4.181614
Fes                       4.148485
Kalavrita                 4.148485
Rome                      4.148485
Laguardia                 4.132681
Tavistock                 4.109524
Ham Tien                  4.072917
Pau                       4.072917
Hobart                    4.072917
Napa                      4.072917
New Orleans               4.072917
Saint-Yrieix-la-Perche    4.072917
Portland                  4.055556
Porlock                   4.055556
Stone                     4.055556
Manchester                4.055556
Singapore                 4.049652
Funchal                   4.047926
Payangan                  4.040385
Ko Samui                  4.040385
Newport -Trefdraeth       4.040385
Hensol                    4.040385
Hoi An                    4.040385
Felin Fach                4.040385
Hanoi                     4.040385
Lisbon                    4.040385
St. Petersburg            4.040385
Feering

In [246]:
df_neighbours = df_neighbours.replace(0,np.nan)
df_prediction = df.loc['ASWh',:].replace(0,np.nan).mean() + df_neighbours.sub(df_neighbours.mean(axis='columns'),axis=0).multiply(sims,axis=0).sum().divide(df_neighbours.multiply(sims,axis=0).sum())
df_prediction.sort_values(ascending=False)

taObjectCity
Gullane                4.180000
Frankfurt              4.177632
Wichita                4.177632
Prague                 4.177632
Chelsea                4.177632
Denver                 4.177632
San Francisco          4.177632
Palma de Mallorca      4.177632
Tortola                4.177632
Imerovigli             4.177632
Cannes                 4.150980
Stratford-upon-Avon    4.150980
Long Itchington        4.150980
Banbury                4.150980
Grasse                 4.150980
Flushing               4.105556
Lima                   4.105556
Bath                   4.103704
Dubai                  4.100000
Aughrim                4.100000
Boyle                  4.100000
Ho Chi Minh City       4.100000
Harrogate              4.071460
Brunswick              4.065376
Baltimore              4.065376
Keene                  4.065376
Round Pond             4.065376
Port Huron             4.065376
Glendale               4.059033
Las Vegas              4.056971
                         ..